# PowerShell: Select-Object with Calculated Properties

Let's **combine properties from multiple objects** using `Select-Object` calculated properties.

> Tip: In Jupyter, pick the **PowerShell** kernel (from .NET Interactive) for these cells to run.


## 1) File owner + file info (Get-ChildItem + Get-Acl)
Combine file metadata with the **owner** pulled from an ACL object.

In [ ]:
Get-ChildItem -Path . -File |
    Select-Object Name, Length,
        @{Name='Owner'; Expression = { (Get-Acl -Path $_.FullName).Owner }}

## 2) Network connections + process metadata (Get-NetTCPConnection + Get-Process)
Map each TCP connection to its owning process and show the process **name** and **CPU** time.

In [ ]:
$procs = Get-Process | Group-Object -Property Id -AsHashTable -AsString
Get-NetTCPConnection -State Established |
    Select-Object LocalAddress, LocalPort, RemoteAddress, RemotePort, OwningProcess,
        @{Name='ProcessName'; Expression = { $procs["$($_.OwningProcess)"].ProcessName }},
        @{Name='CPU';         Expression = { $procs["$($_.OwningProcess)"].CPU }}

## 3) Network adapter + IP configuration (Get-NetAdapter + Get-NetIPConfiguration)
Join adapter rows with IP details using the `InterfaceIndex`.

In [ ]:
$ipByIndex = Get-NetIPConfiguration | Group-Object -Property InterfaceIndex -AsHashTable
Get-NetAdapter |
    Select-Object Name, Status, MacAddress,
        @{Name='IPv4'; Expression = { ($ipByIndex[$_.ifIndex].IPv4Address.IPAddress) -join ', ' }},
        @{Name='IPv6'; Expression = { ($ipByIndex[$_.ifIndex].IPv6Address.IPAddress) -join ', ' }}

## 4) ScheduledTask + ScheduledTaskInfo
Each `ScheduledTask` has a matching `ScheduledTaskInfo`. Use a calculated property to fetch **LastRunTime** and **LastTaskResult**.

In [ ]:
Get-ScheduledTask |
    Select-Object TaskName, State,
        @{Name='LastRunTime';    Expression = { (Get-ScheduledTaskInfo -TaskName $_.TaskName).LastRunTime }},
        @{Name='LastTaskResult'; Expression = { (Get-ScheduledTaskInfo -TaskName $_.TaskName).LastTaskResult }}

## 5) Process + code signing info (Get-Process + Get-AuthenticodeSignature)
Augment processes with the **Publisher** of the executable (if available).

In [ ]:
Get-Process |
    Select-Object Name, Id,
        @{Name='Publisher'; Expression = {
            if ($_.Path) {
                try { (Get-AuthenticodeSignature -FilePath $_.Path).SignerCertificate.Subject }
                catch { $null }
            }
        }}

## 6) Service + owning process details (Get-Service + Get-Process)
Enrich each service with details from its process (when available).

In [ ]:
Get-Service |
    ForEach-Object {
        $p = Get-Process -Id $_.Id -ErrorAction SilentlyContinue
        [pscustomobject]@{
            Service   = $_.Name
            Status    = $_.Status
            PID       = $p.Id
            CPU       = $p.CPU
            StartTime = $p.StartTime
        }
    } |
    Select-Object Service, Status, PID, CPU, StartTime

---
### Handy Patterns
- **Lookup tables**: `Group-Object -AsHashTable` is perfect for fast joins in calculated properties.
- **Per-row queries**: Call another cmdlet inside the expression to attach related info to each row.
- **Formatting**: Use strings and `-f` formatting to present data cleanly.
